In [ ]:
import asyncio
import httpx
import json # Import json module for exception handling


async def fetch_product(client: httpx.AsyncClient, url: str) -> dict:
    response = await client.get(url)
    # Check if the response status is OK and if it contains JSON
    if response.status_code == 200 and 'application/json' in response.headers.get('Content-Type', ''):
        try:
            return response.json()
        except json.JSONDecodeError:
            print(f"Warning: Could not decode JSON from {url}. Response content: {response.text[:200]}...")
            return {}
    else:
        print(f"Warning: Received non-JSON response (Status: {response.status_code}, Content-Type: {response.headers.get('Content-Type')}) from {url}. Content: {response.text[:200]}...")
        return {}


async def scrape_shopify_products(base_url: str, client: httpx.AsyncClient) -> list:
    all_products = []
    page_number = 1

    while True:
        data = await fetch_product(
            client,
            f"{base_url}/products.json?page={page_number}"
        )

        products = data.get("products", [])
        if not products:
            break

        all_products.extend(products)
        page_number += 1

    return all_products


# Run scraping
async def main():
    async with httpx.AsyncClient() as client:
        products = await scrape_shopify_products("https://www.daouk.com", client)

    print(f"Found {len(products)} products 🛍️")

# Fix: Directly await main() when an event loop is already running (e.g., in Colab)
await main()

<html lang="en">
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Verifying your connection...</title>

    <...
Found 0 products 🛍️


In [ ]:
import requests
import json
import time

all_products = []
page_number = 1
page_limit = 250 # Shopify API default limit

while True:
    print(f"Fetching page {page_number}...")
    url = f"https://fattalonline.com/products.json?page={page_number}&limit={page_limit}"

    try:
        res = requests.get(url)
        res.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = res.json()
        products = data.get('products', [])

        all_products.extend(products)

        if len(products) < page_limit:
            print(f"Found {len(products)} products on page {page_number}. Assuming this is the last page.")
            break

        print(f"Found {len(products)} products on page {page_number}.")
        page_number += 1
        time.sleep(0.5) # Be kind to the server

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        break
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {url}. Response content: {res.text[:200]}...")
        break

print(f"\nTotal products scraped: {len(all_products)}")

# Optionally, save to a JSON file
file_path = 'fattalonline_products.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(all_products, f, indent=4)
print(f"All products saved to {file_path}")

Fetching page 1...
Found 250 products on page 1.
Fetching page 2...
Found 250 products on page 2.
Fetching page 3...
Found 250 products on page 3.
Fetching page 4...
Found 250 products on page 4.
Fetching page 5...
Found 250 products on page 5.
Fetching page 6...
Found 250 products on page 6.
Fetching page 7...
Found 250 products on page 7.
Fetching page 8...
Found 250 products on page 8.
Fetching page 9...
Found 250 products on page 9.
Fetching page 10...
Found 250 products on page 10.
Fetching page 11...
Found 250 products on page 11.
Fetching page 12...
Found 250 products on page 12.
Fetching page 13...
Found 250 products on page 13.
Fetching page 14...
Found 99 products on page 14. Assuming this is the last page.

Total products scraped: 3349
All products saved to fattalonline_products.json


In [ ]:
import re
# from bs4 import BeautifulSoup

res = requests.get("https://daouk.com/products/braun-silk-epil-3-3000-epilator-se3-000-purple")
# soup = BeautifulSoup(res.text, 'html.parser')
text = res.text

pattern = re.compile(r'(?i)\b(?:barcode|gtin)\b\s*[:"]?\s*([0-9]{8,14})')
matches = pattern.findall(text)
print(matches)

'<!doctype html>\r\n<html class="no-js" lang="en" dir="ltr">\r\n<head>\r\n\r\n\t<meta charset="utf-8">\r\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\r\n  <meta name="viewport" content="width=device-width, initial-scale=1.0, height=device-height, minimum-scale=1.0"><link rel="shortcut icon" href="//daouk.com/cdn/shop/files/455867063_913187604168819_8885648621369374757_n_1_-Photoroom_2_-Photoroom.png?crop=center&height=48&v=1737464731&width=48" type="image/png" /><title>Braun Silk-Epil 3 3000 Epilator Se3-000 Purple &ndash; Izzat Daouk Lebanon \r</title><meta name="description" content="20 tweezer system Smart Light: Reveals the finest hairs. Soft lift tips 2 Speed Settings Massage rollers Corded Use Included high frequency massage cap, cleaning brush Easy long-lasting smooth skin: removes even short hairs from the root Comfortable epilation: the massage roller cap helps reduce pain sensation Smart li">\n\n<meta property="og:site_name" content="Izzat Daouk Lebanon "

The `products` variable now contains a list of `ShopifyProduct` objects. Let's inspect the first few products to see the structure of the data.

In [ ]:
# Display the total number of products found
print(f"Total products scraped: {len(products)}")

# Display the first product in the list
if products:
    print("\nFirst product:")
    print(products[0].model_dump_json(indent=2))

# Optionally, display the title and price of the first variant of the first product
# if products and products[0].variants:
#     print("\nFirst product's first variant title and price:")
#     print(f"Title: {products[0].variants[0].title}")
#     print(f"Price: {products[0].variants[0].price}")

Total products scraped: 4225

First product:
{
  "id": 8788552188053,
  "title": "Ombres G Eyeshadow Quad",
  "vendor": "Guerlain",
  "tags": [
    "Eye",
    "Eyeshadow",
    "Luxury",
    "Make Up",
    "Make Up Palettes"
  ],
  "variants": [
    {
      "id": 46052464525461,
      "title": "011 Imperial Moon",
      "price": "109.00",
      "available": true,
      "sku": "I045870",
      "featured_image": {
        "id": 43338970136725,
        "src": "https://cdn.shopify.com/s/files/1/0716/5791/4517/files/3346470436558.jpg?v=1748355550",
        "width": 2048,
        "height": 2048
      }
    },
    {
      "id": 46052464558229,
      "title": "214 Exotic Orchid",
      "price": "109.00",
      "available": true,
      "sku": "I045871",
      "featured_image": {
        "id": 43338970202261,
        "src": "https://cdn.shopify.com/s/files/1/0716/5791/4517/files/3346470436510.jpg?v=1748355565",
        "width": 2048,
        "height": 2048
      }
    },
    {
      "id": 4605246

In [ ]:
import json

# Convert Pydantic models to dictionaries
products_dict_list = [product.model_dump() for product in products]

# Save the list of dictionaries to a JSON file
file_path = 'products.json'
with open(file_path, 'w') as f:
    json.dump(products_dict_list, f, indent=4)

print(f"Products data saved to {file_path}")

Products data saved to products.json


In [ ]:
import json
import requests
from bs4 import BeautifulSoup

url = "https://cf.com.lb/recommendations/products?product_id=8984818778261&section_id=template--19295305793685__main"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')


variant_script = soup.find("script", {"data-variant": True, "type": "application/json"})

if variant_script:
    variant_json_str = variant_script.string.strip()
    variant_data = json.loads(variant_json_str)
    print("Extracted Variant JSON:")
    print(json.dumps(variant_data, indent=2))
else:
    print("Variant JSON script not found")

Extracted Variant JSON:
{
  "id": 46696701165717,
  "title": "60ml",
  "option1": "60ml",
  "option2": null,
  "option3": null,
  "sku": "I056771",
  "requires_shipping": true,
  "taxable": true,
  "featured_image": {
    "id": 45046109372565,
    "product_id": 8984818778261,
    "position": 1,
    "created_at": "2025-12-04T17:34:59+03:00",
    "updated_at": "2025-12-04T17:34:59+03:00",
    "alt": null,
    "width": 2048,
    "height": 2048,
    "src": "//cf.com.lb/cdn/shop/files/3386460169080.jpg?v=1764858899",
    "variant_ids": [
      46696701165717
    ]
  },
  "available": true,
  "name": "Original Pour Femme - Eau de Parfum - 60ml",
  "public_title": "60ml",
  "options": [
    "60ml"
  ],
  "price": 7901,
  "weight": 0,
  "compare_at_price": null,
  "inventory_management": "shopify",
  "barcode": "3386460169080",
  "featured_media": {
    "alt": null,
    "id": 36408661246101,
    "position": 1,
    "preview_image": {
      "aspect_ratio": 1.0,
      "height": 2048,
      "width

In [ ]:
def extract_variants_from_html(html: str):
    soup = BeautifulSoup(html, "html.parser")

    variants = []
    scripts = soup.find_all("script", {"type": "application/json", "data-variant": True})

    for script in scripts:
        try:
            json_data = json.loads(script.string.strip())
            variants.append(json_data)
        except Exception as e:
            print("Skipping script due to parsing error:", e)

    return variants


# Example usage
if __name__ == "__main__":
    html = """PASTE YOUR HTML HERE"""
    result = extract_variants_from_html(html)
    print(json.dumps(result, indent=2))

[
  {
    "id": 46696701165717,
    "title": "60ml",
    "option1": "60ml",
    "option2": null,
    "option3": null,
    "sku": "I056771",
    "requires_shipping": true,
    "taxable": true,
    "featured_image": {
      "id": 45046109372565,
      "product_id": 8984818778261,
      "position": 1,
      "created_at": "2025-12-04T17:34:59+03:00",
      "updated_at": "2025-12-04T17:34:59+03:00",
      "alt": null,
      "width": 2048,
      "height": 2048,
      "src": "//cf.com.lb/cdn/shop/files/3386460169080.jpg?v=1764858899",
      "variant_ids": [
        46696701165717
      ]
    },
    "available": true,
    "name": "Original Pour Femme - Eau de Parfum - 60ml",
    "public_title": "60ml",
    "options": [
      "60ml"
    ],
    "price": 7901,
    "weight": 0,
    "compare_at_price": null,
    "inventory_management": "shopify",
    "barcode": "3386460169080",
    "featured_media": {
      "alt": null,
      "id": 36408661246101,
      "position": 1,
      "preview_image": {
     

In [ ]:
import json
from bs4 import BeautifulSoup

def clean_html(html_text: str) -> str:
    """Remove HTML tags and return clean text."""
    if not html_text:
        return ""
    return BeautifulSoup(html_text, "html.parser").get_text(separator=" ").strip()

def parse_shopify_products(data: dict):
    products = data.get("products", [])

    extracted_data = []

    for p in products:
        handle = p.get("handle")
        if handle:
            product_url = f"https://cf.com.lb/products/{handle}"
        else:
            product_url = None
        product_info = {
            "product_id": p.get("id"),
            "title": p.get("title"),
            "handle": product_url,
            "vendor": p.get("vendor"),
            "product_type": p.get("product_type"),
            "body_html_clean": clean_html(p.get("body_html", "")),
            "images": [img.get("src") for img in p.get("images", [])],
            "options": {
                "size": [],
                "color": []
            },
            "variants": []
        }

        # Extract size & color options
        for opt in p.get("options", []):
            name = opt.get("name", "").lower()
            if "size" in name:
                product_info["options"]["size"] = opt.get("values", [])
            if "color" in name:
                product_info["options"]["color"] = opt.get("values", [])

        # Variants
        for v in p.get("variants", []):
            variant_info = {
                "variant_id": v.get("id"),
                "title": v.get("title"),
                "sku": v.get("sku"),
                "available": v.get("available"),
                "price": v.get("price"),
                "grams": v.get("grams"),
                "barcode": v.get("barcode"),
                "image": v.get("featured_image")['src'] if v.get("featured_image") else None
            }
            product_info["variants"].append(variant_info)

        extracted_data.append(product_info)

    return extracted_data


# ========= Example Usage =========

# Load your JSON example
# json_str = """PUT_JSON_HERE"""
# shopify_data = json.loads(products)

# Fix: Pass a dictionary with a 'products' key containing the list of product dictionaries
parsed = parse_shopify_products({"products": products})
parsed[0]
# for prod in parsed:
#     print(json.dumps(prod, indent=2))

In [ ]:
parsed[9]

{'product_id': 8960340131989,
 'title': 'What A Tint! Staining Lip & Cheek Tint',
 'handle': 'https://cf.com.lb/products/what-a-tint-staining-lip-cheek-tint',
 'vendor': 'Essence',
 'product_type': 'Casual',
 'body_html_clean': 'Add a juicy splash of colour to your look with the essence WHAT A TINT! staining lip & cheek tint. The balm texture melts into the skin, leaving a radiant flush on lips and cheeks that looks effortless and feels ultra-light. With its delicious fruity scent and refreshing sensation on application, it’s a treat for all the senses. The practical stick format makes it perfect for quick touch-ups, delivering a fresh and naturally radiant look with a staining effect in seconds.',
 'images': ['https://cdn.shopify.com/s/files/1/0716/5791/4517/files/4059729542540.jpg?v=1762952430',
  'https://cdn.shopify.com/s/files/1/0716/5791/4517/files/4059729542571.jpg?v=1762952448'],
 'options': {'size': [],
  'color': ['10 Raspberry Vibes', '20 Grapefruit Bliss']},
 'variants': [{

In [ ]:
import re

def fill_missing_variant_barcodes(product):
    if "variants" not in product:
        return product

    for variant in product["variants"]:
        # Keep existing barcode if valid
        if variant.get("barcode"):
            continue

        img = variant.get("featured_image") or {}
        src = img.get("src", "") if isinstance(img, dict) else ""

        match = re.search(r'(\d+)\.jpg', src)
        variant["barcode"] = match.group(1) if match else None

    return product


# Apply the fix to all products
for i, product in enumerate(products):
    products[i] = fill_missing_variant_barcodes(product)

print("🎯 Barcode extraction and patching complete!")


🎯 Barcode extraction and patching complete!


In [ ]:
products[0]

{'id': 8788552188053,
 'title': 'Ombres G Eyeshadow Quad',
 'handle': 'ombres-g-eyeshadow-quad',
 'body_html': '<p>Ombres G eyeshadow quads have an exceptional texture made up of intense pigments for long-lasting colour, offering unparalleled depth.</p>\n\n<p>True gems of nature, the Ombres G adorn the eyes with four different finishes: an intense satin, a deep matte, a luminous metallic finish and a radiant pearl.</p>\n\n<p>Violette, Guerlain Creative Director of Make-Up, has imagined generous and balanced palettes, which allow you to play with shades, textures and finishes, creating a host of effects to endlessly transform your eyes both day and night.</p>',
 'published_at': '2025-12-08T17:34:57+03:00',
 'created_at': '2025-05-27T12:05:32+03:00',
 'updated_at': '2025-12-09T05:21:24+03:00',
 'vendor': 'Guerlain',
 'product_type': 'Luxury',
 'tags': ['Eye', 'Eyeshadow', 'Luxury', 'Make Up', 'Make Up Palettes'],
 'variants': [{'id': 46052464525461,
   'title': '011 Imperial Moon',
   'o

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

BASE_URL = "https://cf.com.lb"

def get_val(obj, key, default=None):
    if isinstance(obj, dict):
        return obj.get(key, default)
    return getattr(obj, key, default)


for product in products:
    product_id = get_val(product, "id")
    if not product_id:
        continue

    print(f"🔍 Getting barcode for product: {get_val(product,'title')} ({product_id})")

    url = f"{BASE_URL}/recommendations/products?product_id={product_id}&section_id=template--19295305793685__main"

    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")

        variant_script = soup.find("script", {"data-variant": True, "type": "application/json"})
        if not variant_script:
            print("  ❌ No variant script found")
            continue

        try:
            raw_data = json.loads(variant_script.text)

            # Normalize into list ALWAYS
            if isinstance(raw_data, dict):
                variant_json = [raw_data]
            elif isinstance(raw_data, list):
                variant_json = raw_data
            else:
                print("⚠ Unexpected JSON structure")
                continue

        except json.JSONDecodeError:
            print("⚠ Failed to decode JSON")
            continue

        for variant in get_val(product, "variants", []):
            v_id = get_val(variant, "id")

            match = next((v for v in variant_json
                          if isinstance(v, dict) and v.get("id") == v_id), None)

            barcode = match.get("barcode") if match else None

            # Save back into object
            if barcode:
                if isinstance(variant, dict):
                    variant["barcode"] = barcode
                else:
                    setattr(variant, "barcode", barcode)

                print(f"  ✔ {get_val(variant,'title')} → {barcode}")
            else:
                print(f"  ❌ {get_val(variant,'title')} barcode missing")


        time.sleep(1)

    except Exception as exc:
        print(f"Error: {exc}")

print("\n🎉 All barcodes processed!")


🔍 Getting barcode for product: Ombres G Eyeshadow Quad (8788552188053)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dolce Vita - Eau de Toilette (8989277946005)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dune Pour Homme - Eau de Toilette (8989276274837)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Eau Sauvage - Eau de Toilette (8989275062421)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dior Homme After Shave Lotion (8989261627541)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Dior Homme Deodorant Stick (8989261398165)
Error: "ShopifyVariant" object has no field "barcode"
🔍 Getting barcode for product: Million Gold For Her Gift Set (8867229597845)
  ❌ No variant script found
🔍 Getting barcode for product: Invictus Victory Elixir Gift Set (8867229499541)
  ❌ No variant script found
🔍 Getti

KeyboardInterrupt: 

In [ ]:
import asyncio
import httpx
import json
import requests
from bs4 import BeautifulSoup
import time

BASE_URL = "https://cf.com.lb"


# ----------------------------------------
# 🛍️ Fetch products from Shopify
# ----------------------------------------
async def fetch_page(client, url):
    res = await client.get(url)
    return res.json()


async def scrape_products():
    all_products = []
    page = 1

    async with httpx.AsyncClient() as client:
        while True:
            url = f"{BASE_URL}/products.json?page={page}"
            print(f"Fetching: {url}")

            data = await fetch_page(client, url)
            products = data.get("products", [])

            if not products:
                break

            all_products.extend(products)
            page += 1

    print(f"✔ Total products fetched: {len(all_products)}")
    return all_products


# ----------------------------------------
# 📦 Fetch barcode for each product variant
# ----------------------------------------
def add_barcodes(products):

    for p in products:
        product_id = p.get("id")
        if not product_id:
            continue

        print(f"\n🔍 {p.get('title')} ({product_id})")

        url = f"{BASE_URL}/recommendations/products?product_id={product_id}&section_id=template--19295305793685__main"

        try:
            html = requests.get(url).text
            soup = BeautifulSoup(html, "html.parser")

            script_tag = soup.find("script", {"data-variant": True, "type": "application/json"})
            if not script_tag:
                print("  ❌ No variant script")
                continue

            raw_json = json.loads(script_tag.text)

            # Normalize JSON to list
            variant_data = raw_json if isinstance(raw_json, list) else [raw_json]

            for v in p.get("variants", []):
                v_id = v.get("id")
                match = next((x for x in variant_data if x.get("id") == v_id), None)

                barcode = match.get("barcode") if match else None
                v["barcode"] = barcode

                if barcode:
                    print(f"  ✔ {v.get('title')} → {barcode}")
                else:
                    print(f"  ❌ {v.get('title')} no barcode")

        except Exception as e:
            print(f"  ⚠ Error: {e}")

        time.sleep(0.7)  # Shopify safety / avoid blocking

    print("\n🎉 Barcode scraping completed")


# ----------------------------------------
# 🚀 Run Sequence
# ----------------------------------------
# ----------------------------------------
# 🚀 Run Sequence on Google Colab
# ----------------------------------------
import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
products = loop.run_until_complete(scrape_products())

add_barcodes(products)


Streaming output truncated to the last 5000 lines.

🔍 Gentleman Intense - Eau de Toilette (8798186930325)
  ✔ 60ml → 3274872422995
  ❌ 100ml no barcode

🔍 Reserve Privee Gift Set (8798186471573)
  ❌ No variant script

🔍 Gentleman - Eau de Parfum Boisee (8798186274965)
  ✔ 60ml → 3274872425002
  ❌ 100ml no barcode

🔍 Gentleman - Eau de Toilette (8798186111125)
  ✔ 60ml → 3274872424999
  ❌ 100ml no barcode

🔍 Gentleman - Eau de Parfum (8798185947285)
  ✔ 50ml → 3274872424982
  ❌ 100ml no barcode

🔍 Gentleman - Eau de Cologne (8798185783445)
  ✔ 50ml → 3274872382374

🔍 Gentlman Hair and Body Shower Gel (8798185619605)
  ✔ 150ml → 3274872368811

🔍 Dahlia Divin Nude - Eau de Parfum (8798185422997)
  ✔ 50ml → 3274872350830

🔍 Dahlia Divin Eau Initial - Eau de Toilette (8798185193621)
  ✔ 50ml → 3274872365933

🔍 Pour Homme - Eau De Toilette (8796667510933)
  ✔ 100ml → 3274870303166

🔍 Le Parfum Royal - Eau de Parfum (8796753854613)
  ✔ 50ml → 7640233340080
  ❌ 90ml no barcode

🔍 Elie Saab L'H

In [ ]:
import json

# Convert Pydantic models to dictionaries
# products_dict_list = [product.model_dump() for product in products]

# Save the list of dictionaries to a JSON file
file_path = 'products-with-barcode-raw.json'
with open(file_path, 'w') as f:
    json.dump(parsed, f, indent=4)

print(f"Products data saved to {file_path}")

Products data saved to products-with-barcode-raw.json


In [ ]:
products[0]

{'id': 8788552188053,
 'title': 'Ombres G Eyeshadow Quad',
 'handle': 'ombres-g-eyeshadow-quad',
 'body_html': '<p>Ombres G eyeshadow quads have an exceptional texture made up of intense pigments for long-lasting colour, offering unparalleled depth.</p>\n\n<p>True gems of nature, the Ombres G adorn the eyes with four different finishes: an intense satin, a deep matte, a luminous metallic finish and a radiant pearl.</p>\n\n<p>Violette, Guerlain Creative Director of Make-Up, has imagined generous and balanced palettes, which allow you to play with shades, textures and finishes, creating a host of effects to endlessly transform your eyes both day and night.</p>',
 'published_at': '2025-12-08T17:34:57+03:00',
 'created_at': '2025-05-27T12:05:32+03:00',
 'updated_at': '2025-12-09T05:21:24+03:00',
 'vendor': 'Guerlain',
 'product_type': 'Luxury',
 'tags': ['Eye', 'Eyeshadow', 'Luxury', 'Make Up', 'Make Up Palettes'],
 'variants': [{'id': 46052464525461,
   'title': '011 Imperial Moon',
   'o

In [ ]:
import pandas as pd

def flatten_products(products: list[dict]) -> pd.DataFrame:
    rows = []

    for product in products:
        product_id = product.get("product_id")
        product_title = product.get("title", "")
        vendor = product.get("vendor", "")
        product_type = product.get("product_type", "")
        handle = product.get("handle", "")
        description = product.get("body_html_clean", "")

        # SAFELY resolve default image
        images = product.get("images") or []
        default_image = images[0] if images else None

        # Prepare default values
        option_colors = []
        option_sizes = []

        # Options can be dict OR list
        options = product.get("options", {})

        if isinstance(options, dict):
            option_colors = options.get("color", [])
            option_sizes = options.get("size", [])
        elif isinstance(options, list):
            for opt in options:
                name = (opt.get("name") or "").lower()
                if name == "color":
                    option_colors = opt.get("values", [])
                if name == "size":
                    option_sizes = opt.get("values", [])

        for variant in product.get("variants", []):
            variant_title = variant.get("title") or product_title

            # Replace default title
            if variant_title.lower() == "default title":
                variant_title = product_title

            # Fallback to product image
            image = variant.get("image") or default_image

            color = variant_title if variant_title in option_colors else None
            size = variant_title if variant_title in option_sizes else None

            rows.append({
                "product_id": product_id,
                "product_title": product_title,
                "vendor": vendor,
                "product_type": product_type,
                "handle": handle,
                "description": description,
                "variant_id": variant.get("variant_id"),
                "variant_title": variant_title,
                "sku": variant.get("sku"),
                "price": variant.get("price"),
                "color": color,
                "size": size,
                "available": variant.get("available"),
                "grams": variant.get("grams"),
                "barcode": variant.get("barcode"),
                "image": image
            })

    return pd.DataFrame(rows)



df = flatten_products(products)

# Display as table in Colab (Excel-ready)
df

,product_id,product_title,vendor,product_type,handle,description,variant_id,variant_title,sku,price,color,size,available,grams,barcode,image
0,8788552188053,Ombres G Eyeshadow Quad,Guerlain,Luxury,https://cf.com.lb/products/ombres-g-eyeshadow-...,Ombres G eyeshadow quads have an exceptional t...,46052464525461,011 Imperial Moon,I045870,109.00,011 Imperial Moon,None,True,0,3346470436558,https://cdn.shopify.com/s/files/1/0716/5791/45...
1,8788552188053,Ombres G Eyeshadow Quad,Guerlain,Luxury,https://cf.com.lb/products/ombres-g-eyeshadow-...,Ombres G eyeshadow quads have an exceptional t...,46052464558229,214 Exotic Orchid,I045871,109.00,214 Exotic Orchid,None,True,0,3346470436510,https://cdn.shopify.com/s/files/1/0716/5791/45...
2,8788552188053,Ombres G Eyeshadow Quad,Guerlain,Luxury,https://cf.com.lb/products/ombres-g-eyeshadow-...,Ombres G eyeshadow quads have an exceptional t...,46052464459925,258 Wild Nudes,I049472,109.00,258 Wild Nudes,None,True,0,3346470439634,https://cdn.shopify.com/s/files/1/0716/5791/45...
3,8788552188053,Ombres G Eyeshadow Quad,Guerlain,Luxury,https://cf.com.lb/products/ombres-g-eyeshadow-...,Ombres G eyeshadow quads have an exceptional t...,46052464590997,360 Mystic Peacock,I045872,109.00,360 Mystic Peacock,None,True,0,3346470436503,https://cdn.shopify.com/s/files/1/0716/5791/45...
4,8788552188053,Ombres G Eyeshadow Quad,Guerlain,Luxury,https://cf.com.lb/products/ombres-g-eyeshadow-...,Ombres G eyeshadow quads have an exceptional t...,46726528794773,520 Stellar Glow,I056842,115.00,520 Stellar Glow,None,False,0,3346470621909,https://cdn.shopify.com/s/files/1/0716/5791/45...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7978,8775370997909,Hypoallergenic Fresh Mat Pocket Powder,Bell HYPOAllergenic,Casual,https://cf.com.lb/products/hypoallergenic-fres...,Gives the skin fresh and rested look. Due to...,46369745272981,06,I042174,11.00,06,None,True,0,5902082539285,https://cdn.shopify.com/s/files/1/0716/5791/45...
7979,8775370965141,Hypoallergenic Fresh Mat Liquid Lipstick,Bell HYPOAllergenic,Casual,https://cf.com.lb/products/hypoallergenic-fres...,Its light and long-lasting formula creates a m...,46369830928533,01 Daisy,I042152,6.50,01 Daisy,None,False,0,5902082540298,https://cdn.shopify.com/s/files/1/0716/5791/45...
7980,8775370965141,Hypoallergenic Fresh Mat Liquid Lipstick,Bell HYPOAllergenic,Casual,https://cf.com.lb/products/hypoallergenic-fres...,Its light and long-lasting formula creates a m...,45999244705941,02 Orchid,I042153,6.50,02 Orchid,None,True,0,5902082540304,https://cdn.shopify.com/s/files/1/0716/5791/45...
7981,8775370965141,Hypoallergenic Fresh Mat Liquid Lipstick,Bell HYPOAllergenic,Casual,https://cf.com.lb/products/hypoallergenic-fres...,Its light and long-lasting formula creates a m...,45999244738709,03 Dahlia,I042154,6.50,03 Dahlia,None,False,0,5902082540311,https://cdn.shopify.com/s/files/1/0716/5791/45...
